In [3]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Fetching 29 files:   0%|          | 0/29 [00:00<?, ?it/s]

In [ ]:

from qdrant_search import QdrantSearchEngine
import asyncio

# Initialize search engine
search = QdrantSearchEngine()


In [23]:


# Search parameters
query = "курсор"
meeting_ids = ["8ff59e4c-7cf3-444d-88bc-c6b25392e3e2"]
limit = 10
min_score = -1.

# Perform search
results = await search.search_transcripts(
    query_text=query,
    meeting_ids=meeting_ids,
    limit=limit,
    min_score=min_score,
    length_penalty=1.,
    min_length=1,
    max_length=25
)


Model device: cuda:3


In [24]:
results

,content,speaker,timestamp,similarity,meeting_id,indices,vector_score,text_score,length
0,"чего? Ланч курсор? Вот там, где плюсик.",Sergey Ryabenko,2024-11-06T15:47:33.224000Z,0.692628,8ff59e4c-7cf3-444d-88bc-c6b25392e3e2,"[36, 37]",0.692628,0.377964,41
1,— На фронте вот он есть.,Sergey Ryabenko,2024-11-06T15:46:44.194000Z,0.595597,8ff59e4c-7cf3-444d-88bc-c6b25392e3e2,[25],0.595597,0.000000,25
2,"Курсор. Вот, можно нажимаешь, и все. Подожд...",Dmitriy Grankin,2024-11-06T15:47:26.304000Z,0.558699,8ff59e4c-7cf3-444d-88bc-c6b25392e3e2,"[33, 34, 35]",0.558699,0.333333,55
3,Ты мне выслал. выслал я сейчас попробую в эт...,Sergey Ryabenko,2024-11-06T15:48:03.680000Z,0.546222,8ff59e4c-7cf3-444d-88bc-c6b25392e3e2,"[48, 49]",0.546222,0.000000,97
4,Что конкретно?,Sergey Ryabenko,2024-11-06T15:46:49.644000Z,0.381612,8ff59e4c-7cf3-444d-88bc-c6b25392e3e2,[27],0.381612,0.000000,15
5,"просто, вот у тебя же ты курсор открыл. Не, ...",Dmitriy Grankin,2024-11-06T15:50:30.876000Z,0.340739,8ff59e4c-7cf3-444d-88bc-c6b25392e3e2,"[70, 71]",0.000000,0.340739,59
6,"Плюсик, да.",Dmitriy Grankin,2024-11-06T15:47:36.864000Z,0.303107,8ff59e4c-7cf3-444d-88bc-c6b25392e3e2,[38],0.303107,0.000000,12
7,Как там...,Sergey Ryabenko,2024-11-06T15:51:38.352000Z,0.281093,8ff59e4c-7cf3-444d-88bc-c6b25392e3e2,[79],0.281093,0.000000,11
8,"А, я понял.",Dmitriy Grankin,2024-11-06T15:51:37.492000Z,0.270721,8ff59e4c-7cf3-444d-88bc-c6b25392e3e2,[78],0.270721,0.000000,12
9,пишешь в поиске открываешь его и здесь три кн...,Dmitriy Grankin,2024-11-06T15:47:49.244000Z,0.261261,8ff59e4c-7cf3-444d-88bc-c6b25392e3e2,[40],0.000000,0.261261,88


In [ ]:


query = "глобальный экономический кризис"
meeting_ids = ["8ff59e4c-7cf3-444d-88bc-c6b25392e3e2"]

results = await search.search_transcripts_with_context(
    query_text=query,
    meeting_ids=meeting_ids


In [1]:
from qdrant_client import AsyncQdrantClient
from sentence_transformers import SentenceTransformer
import asyncio

# Initialize
client = AsyncQdrantClient("localhost", port=6333)
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2', device='cuda:3')
collection = "discussion_points"


/home/dima/anaconda3/envs/langchain/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
from qdrant_client.models import Filter, FieldCondition, MatchValue, MatchText, SearchParams,MatchAny

In [37]:
self = QdrantSearchEngine()

/home/dima/anaconda3/envs/langchain/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [67]:
from fastembed import SparseTextEmbedding

In [68]:
self.sparse_model = SparseTextEmbedding(model_name="Qdrant/bm25")

Fetching 29 files:   0%|          | 0/29 [00:00<?, ?it/s]

In [74]:
from typing import List
import numpy as np
import pandas as pd

In [75]:
async def calculate_sparse_similarity(self, query_text: str, content: str) -> float:
    # Get sparse embeddings
    embeddings = list(self.sparse_model.embed([query_text, content]))
    if len(embeddings) != 2:
        return 0.0
        
    query_emb, content_emb = embeddings
    
    # Find common indices
    common_indices = np.intersect1d(query_emb.indices, content_emb.indices)
    if len(common_indices) == 0:
        return 0.0
        
    # Get positions of common indices
    query_pos = np.searchsorted(query_emb.indices, common_indices)
    content_pos = np.searchsorted(content_emb.indices, common_indices)
    
    # Calculate dot product for matching terms
    similarity = np.sum(query_emb.values[query_pos] * content_emb.values[content_pos])
    
    # Normalize by document lengths
    norm = np.sqrt(np.sum(query_emb.values**2) * np.sum(content_emb.values**2))
    if norm > 0:
        similarity /= norm
        
    return float(similarity)

async def search_transcripts(self, query_text: str, meeting_ids: List[str] = None, limit: int = 20, min_score: float = 0.3):
    # Base filter conditions
    must_conditions = [
        FieldCondition(key="vector_type", match=MatchValue(value="transcript"))
    ]
    if meeting_ids:
        must_conditions.append(
            FieldCondition(key="meeting_id", match=MatchAny(any=meeting_ids))
        )

    # 1. Vector search
    query_vector = await self.encode_text(query_text)
    vector_results = await self.client.search(
        collection_name=self.collection_name,
        query_vector=query_vector.tolist(),
        query_filter=Filter(must=must_conditions),
        limit=limit,
        score_threshold=min_score,
        search_params=SearchParams(hnsw_ef=128),
        with_payload=True
    )
    
    # 2. Get all meeting content
    scroll_response = await self.client.scroll(
        collection_name=self.collection_name,
        scroll_filter=Filter(must=must_conditions),
        limit=10000,
        with_payload=True
    )
    all_chunks = scroll_response[0]

    # 3. Combine and score results
    vector_scores = {hit.payload['content']: hit.score for hit in vector_results}
    
    results = []
    for chunk in all_chunks:
        content = chunk.payload['content']
        
        # Get vector score if exists, else 0
        vector_score = vector_scores.get(content, 0.0)
        
        # Calculate BM25-like text similarity
        text_score = await self.calculate_sparse_similarity(query_text, content)
        
        # Combined score (weighted combination)
        final_score = max(vector_score, text_score)
        
        if final_score >= min_score:
            results.append({
                "content": content,
                "speaker": chunk.payload.get("speaker", ""),
                "timestamp": chunk.payload.get("timestamp", ""),
                "similarity": final_score,
                "meeting_id": chunk.payload.get("meeting_id", ""),
                "indices": chunk.payload.get("indices", []),
                "vector_score": vector_score,
                "text_score": text_score
            })
    
    # Sort by score and limit results
    results.sort(key=lambda x: x['similarity'], reverse=True)
    return pd.DataFrame(results[:limit])

In [85]:
query = "разработчик который работает с сервером"
meeting_ids = ["9211c3e2-6e25-470c-8daf-d317d6555b34"]
limit = 10
min_score = 0.4

In [86]:
await self.search_transcripts(query, meeting_ids, limit, min_score)

Model device: cuda:3


,content,speaker,timestamp,similarity,meeting_id,indices
0,"от сервера, если что-то случилось.",Sergey Ryabenko,2024-11-01T13:46:12.884000Z,0.554024,9211c3e2-6e25-470c-8daf-d317d6555b34,[21]
1,...проблем с сервером и нагрузкой.,Sergey Ryabenko,2024-11-01T13:46:01.644000Z,0.546312,9211c3e2-6e25-470c-8daf-d317d6555b34,[18]
2,"Поэтому я тебе говорю, тебе нужен сервис, а не просто вот DevOps. Вот мое мнение. Ну, более...",Sergey Ryabenko,2024-11-01T13:47:11.734000Z,0.415253,9211c3e2-6e25-470c-8daf-d317d6555b34,"[29, 30, 31]"


In [58]:
must_conditions = [
    FieldCondition(key="vector_type", match=MatchValue(value="transcript"))
]
if meeting_ids:
    must_conditions.append(
        FieldCondition(key="meeting_id", match=MatchAny(any=meeting_ids))
    )

In [59]:
from qdrant_search import QdrantSearchEngine

In [60]:
query_text = query

In [61]:

# 1. Vector search
query_vector = await self.encode_text(query_text)
vector_results = await self.client.search(
    collection_name=self.collection_name,
    query_vector=query_vector.tolist(),
    query_filter=Filter(must=must_conditions),
    limit=limit,
    score_threshold=min_score,
    search_params=SearchParams(hnsw_ef=128),
    with_payload=True
)

Model device: cuda:3


In [62]:

# 2. Get all meeting content
scroll_response = await self.client.scroll(
    collection_name=self.collection_name,
    scroll_filter=Filter(must=must_conditions),
    limit=10000,
    with_payload=True
)
all_chunks = scroll_response[0]

In [63]:

# 3. Combine and score results
vector_scores = {hit.payload['content']: hit.score for hit in vector_results}
query_lower = query_text.lower()
query_words = set(query_lower.split())

results = []
for chunk in all_chunks:
    content = chunk.payload['content']
    content_lower = content.lower()
    content_words = set(content_lower.split())
    
    # Calculate text match score with multiple factors
    text_match_score = 0.0
    
    # Exact phrase match (highest boost)
    if query_lower in content_lower:
        text_match_score += 0.3
    
    # Word-level matching
    word_overlap = len(query_words.intersection(content_words))
    if word_overlap > 0:
        # Normalize by query length for more accurate scoring
        text_match_score += 0.1 * (word_overlap / len(query_words))
    
    # Get vector score if exists, else 0
    vector_score = vector_scores.get(content, 0.0)
    
    # Combined score (vector similarity + text boost)
    final_score = max(vector_score, text_match_score)
    
    if final_score >= min_score:
        results.append({
            "content": content,
            "speaker": chunk.payload.get("speaker", ""),
            "timestamp": chunk.payload.get("timestamp", ""),
            "similarity": final_score,
            "meeting_id": chunk.payload.get("meeting_id", ""),
            "indices": chunk.payload.get("indices", []),
            "vector_score": vector_score,
            "text_score": text_match_score
        })

In [64]:
import pandas as pd

In [65]:
pd.options.display.max_colwidth = 1000

In [66]:
pd.DataFrame(results)

,content,speaker,timestamp,similarity,meeting_id,indices,vector_score,text_score
0,"Поэтому я тебе говорю, тебе нужен сервис, а не просто вот DevOps. Вот мое мнение. Ну, более...",Sergey Ryabenko,2024-11-01T13:47:11.734000Z,0.541408,9211c3e2-6e25-470c-8daf-d317d6555b34,"[29, 30, 31]",0.541408,0.3


/home/dima/anaconda3/envs/langchain/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [41]:

# Search parameters
query = "глобальный"
meeting_ids = ["8ff59e4c-7cf3-444d-88bc-c6b25392e3e2"]
limit = 10
min_score = 0.4

must_conditions = [
    FieldCondition(key="vector_type", match=MatchValue(value="transcript")),
    FieldCondition(key="meeting_id", match=MatchAny(any=meeting_ids))
]


query_vector = model.encode(query)

results = await client.search(
    collection_name=collection,
    query_vector=query_vector.tolist(),
    query_filter=Filter(
        must=must_conditions
    ),
    limit=limit,
    score_threshold=min_score,
    search_params=SearchParams(hnsw_ef=128),
    with_payload=True
)

In [44]:
response = await client.scroll(
            collection_name=collection,
            scroll_filter=Filter(must=must_conditions),
            limit=10000000,
            with_payload=True
        )

In [45]:
response

([Record(id='01552cd4-8dee-45e7-b750-52d1a7cd18ff', payload={'meeting_id': '8ff59e4c-7cf3-444d-88bc-c6b25392e3e2', 'content': ' Плюсик, да.', 'speaker': 'Dmitriy Grankin', 'timestamp': '2024-11-06T15:47:36.864000Z', 'indices': [38], 'vector_type': 'transcript', 'vector_source': 'transcript', 'chunk_index': 23}, vector=None, shard_key=None, order_value=None),
  Record(id='023d8a69-b4ad-47c7-8541-9546fdfd89de', payload={'meeting_id': '8ff59e4c-7cf3-444d-88bc-c6b25392e3e2', 'content': ' пишешь в поиске открываешь его и здесь три кнопки launch vs код launch курсор и connect', 'speaker': 'Dmitriy Grankin', 'timestamp': '2024-11-06T15:47:49.244000Z', 'indices': [40], 'vector_type': 'transcript', 'vector_source': 'transcript', 'chunk_index': 25}, vector=None, shard_key=None, order_value=None),
  Record(id='059620e8-514e-48ee-8385-c7170bf327c7', payload={'meeting_id': '8ff59e4c-7cf3-444d-88bc-c6b25392e3e2', 'content': ' Согласен. Так, ну...  Так, что у нас вообще, какой...  Трэдс чат эриа и ми

In [37]:
results

[ScoredPoint(id='f7ada829-b128-415c-b89d-b511f0808842', version=61, score=0.4828009, payload={'meeting_id': '8ff59e4c-7cf3-444d-88bc-c6b25392e3e2', 'content': ' Что конкретно?', 'speaker': 'Sergey Ryabenko', 'timestamp': '2024-11-06T15:46:49.644000Z', 'indices': [27], 'vector_type': 'transcript', 'vector_source': 'transcript', 'chunk_index': 15}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id='73898a0b-a2fa-4ef0-bdd1-3a813ba5ac58', version=61, score=0.4711425, payload={'meeting_id': '8ff59e4c-7cf3-444d-88bc-c6b25392e3e2', 'content': ' Как там...', 'speaker': 'Sergey Ryabenko', 'timestamp': '2024-11-06T15:51:38.352000Z', 'indices': [79], 'vector_type': 'transcript', 'vector_source': 'transcript', 'chunk_index': 42}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id='2dba0406-fbbe-42a8-a13a-6e4eb71272de', version=61, score=0.45968503, payload={'meeting_id': '8ff59e4c-7cf3-444d-88bc-c6b25392e3e2', 'content': ' Так.', 'speaker': 'TBD', 'timestamp': '2024-11-0

In [ ]:

# Process and boost scores
enhanced_results = []
for hit in results:
    content = hit.payload["content"].lower()
    text_match_score = 0
    
    if query.lower() in content:
        text_match_score += 0.3
    
    word_overlap = len(set(query.lower().split()) & set(content.split()))
    text_match_score += 0.1 * word_overlap
    
    enhanced_results.append({
        "content": hit.payload["content"],
        "speaker": hit.payload["speaker"],
        "similarity": hit.score + text_match_score,
        "meeting_id": hit.payload["meeting_id"]
    })

return sorted(enhanced_results, key=lambda x: x["similarity"], reverse=True)



In [20]:
import pandas as pd

In [21]:
df = pd.DataFrame(results)
df

""
